In [ ]:
import openai
from openai import OpenAI

from google.colab import drive
import gc

import json
import os
import io
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
import time
import datetime
from random import random

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk import ne_chunk
from nltk.tree import Tree
from nltk import RegexpParser
from nltk import word_tokenize
from nltk import pos_tag



# Download the necessary NLTK data files
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker')
nltk.download('maxent_ne_chunker_tab')
nltk.download('words')
nltk.download('stopwords')
nltk.download('treebank')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker_tab.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already

True

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util
from sklearn.decomposition import PCA

from wordcloud import WordCloud

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
from google.colab import userdata
api_key = userdata.get('openaikey')

In [ ]:
client = OpenAI(api_key=api_key)

In [ ]:
# Mount google drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd ./drive/My Drive/SUNY_Poly_DSA598/

/content/drive/My Drive/SUNY_Poly_DSA598


In [ ]:
!ls -a

base_model_eval.ipynb  datasets  GPT_4o_mini_results


In [ ]:

def load_jsonl(file_path):
    """Loads a JSON Lines file into a list of Python objects."""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:  # Specify encoding for safety
        for line in f:
            data.append(json.loads(line))  # Parse each line individually
    return data

## LIAR Base Accuracy and Fine Tuning

### LIAR Conversion to JSONL

In [ ]:
liar_path = './datasets/LIAR/'

In [ ]:
!pwd

/content/drive/MyDrive/SUNY_Poly_DSA598


In [ ]:

liar_label_mapping = {
    "true": "1",
    "mostly-true": "1",
    "half-true": "1",
    "false": "0",
    "pants-fire": "0",
    "barely-true": "0"
}

def convert_liar_tsv_to_jsonl(tsv_filepath, jsonl_filepath):
    """
    Converts a LIAR TSV file to a JSONL file for OpenAI fine-tuning.

    Args:
        tsv_filepath (str): Path to the input TSV file.
        jsonl_filepath (str): Path to the output JSONL file.
    """
    df = pd.read_csv(tsv_filepath, sep='\t', header=None)
    data_list = []

    for index, row in df.iterrows():
        label = row[1]
        statement = row[2]

        mapped_label = liar_label_mapping.get(label) # Use .get() for safety in case of unknown labels

        ########## VARIABLES – SUBJECTIVE PROMPTS ##########
        sys_msg = "You are a fact checking classifier. Classify the truthfulness of the text: '1' if the statement is true, '0' if the statement is false"
        user_msg = "Classify the truthfulness of the following text out of the following categories: '1' if the statement is true, '0' if the statement is false. Do not use any other labels:\n\n" + statement
        assistant_msg = mapped_label
        ###################################################

        if mapped_label: # Only include data points with mapped labels
          data_list.append({
            "messages": [
              {"role": "system", "content": sys_msg},
              {"role": "user", "content": user_msg},
              {"role": "assistant", "content": assistant_msg}
            ]
			})

    with open(jsonl_filepath, 'w') as f:
        for data_item in data_list:
            json.dump(data_item, f)
            f.write('\n') # Write each JSON object on a new line


# Example usage to convert train, valid, and test sets:
convert_liar_tsv_to_jsonl(liar_path + 'train.tsv', liar_path + 'liar_train_user_prompt.jsonl')
convert_liar_tsv_to_jsonl(liar_path + 'valid.tsv', liar_path + 'liar_valid_user_prompt.jsonl')
convert_liar_tsv_to_jsonl(liar_path + 'test.tsv', liar_path + 'liar_test_user_prompt.jsonl')

print("LIAR data conversion to JSONL completed.")

LIAR data conversion to JSONL completed.


In [ ]:
liar_train = load_jsonl(f'{liar_path}liar_train_user_prompt.jsonl')
liar_valid = load_jsonl(f'{liar_path}liar_valid_user_prompt.jsonl')
liar_test = load_jsonl(f'{liar_path}liar_test_user_prompt.jsonl')

In [ ]:
len(liar_train)

10240

### Check GPT-4o-mini's base fact-checking capabilities on LIAR (training set examples)

In [ ]:
# Print the first few  nested objects
for item, i in zip(liar_train, range(5)):
  for key in (item.keys()):
    print(key + ': ' + str(liar_train[i][key]))

messages: [{'role': 'system', 'content': "You are a fact checking classifier. Classify the truthfulness of the text out of the following categories: '1' (if the claim is true), '0' (if the claim is false). Do not use any other labels."}, {'role': 'user', 'content': 'Says the Annies List political group supports third-trimester abortions on demand.'}, {'role': 'assistant', 'content': '0'}]
messages: [{'role': 'system', 'content': "You are a fact checking classifier. Classify the truthfulness of the text out of the following categories: '1' (if the claim is true), '0' (if the claim is false). Do not use any other labels."}, {'role': 'user', 'content': 'When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.'}, {'role': 'assistant', 'content': '1'}]
messages: [{'role': 'system', 'content': "You are a fact checking classifier. Classify the truthfulness of the text out of the following categories: '1' (if 

In [ ]:


# # Load all the files starting with 'meta' in the GPT_4o_mini_results directory and run tokenize_text on the appropriate columns (this was a mistake in the original code)
'''
for file in os.listdir('GPT_4o_mini_results/liar_test'):
    if file.startswith('meta'):
        df = pd.read_csv(os.path.join('GPT_4o_mini_results', file))
        df['usr_msg_num_tokens'] = df['user_msg'].apply(lambda x: tokenize_text(x)[0])
        df['usr_msg_avg_wrd_len'] = df['user_msg'].apply(lambda x: tokenize_text(x)[1])
        df['sys_msg_num_tokens'] = df['sys_msg'].apply(lambda x: tokenize_text(x)[0])
        df['sys_msg_avg_wrd_len'] = df['sys_msg'].apply(lambda x: tokenize_text(x)[1])

        df.to_csv(os.path.join('GPT_4o_mini_results/liar_test', file), index=False)

# Load all the files starting with 'results' in the GPT_4o_mini_results directory and run tokenize_text on the appropriate columns
for file in os.listdir('GPT_4o_mini_results'):
    if file.startswith('results'):
        df = pd.read_csv(os.path.join('GPT_4o_mini_results', file))
        df['claim_num_tokens'] = df['claim'].apply(lambda x: tokenize_text(x)[0])
        df['claim_avg_wrd_len'] = df['claim'].apply(lambda x: tokenize_text(x)[1])
        df.to_csv(os.path.join('GPT_4o_mini_results', file), index=False)

'''

In [ ]:
def fact_check_claim(claim, temp, prompt):

  chat_completion = client.chat.completions.create(
		  messages=[
				{
					"role": "system",
				  "content": prompt['system_message']
				},
				{
				  "role": "user",
				  "content": prompt['user_message'] + "\n\n" + claim
				},
		  ],
		temperature=temp,
		model="gpt-4o-mini-2024-07-18",
		)

  return chat_completion.choices[0].message.content

# Function to tokenize text and return number of tokens and semantic complexity
def tokenize_text(text):
    """
    Tokenizes the input text and returns the number of tokens and semantic complexity.
    """
    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Calculate the number of tokens
    num_tokens = len(tokens)

    # Calculate semantic complexity using a simple heuristic (e.g., average word length)
    avg_word_length = sum(len(word) for word in tokens) / num_tokens if num_tokens > 0 else 0

    return num_tokens, avg_word_length


def run_gpt4o_raw(df, usr_msg, sys_msg, temp, n, rand_state):
    print(f"Prompting GPT with {n} claims, and the following parameters:")
    print(f"Temperature: {temp}")
    print(f"Random State: {rand_state}")

    # Preprocess the dataframe
    df = df[['messages']]

    df['claim'] = df['messages'].apply(lambda x: x[1]['content'])
    df['label'] = df['messages'].apply(lambda x: x[2]['content'])

    df = df[['claim', 'label']]

    df = df.sample(frac=1, random_state=rand_state).reset_index(drop=True)
    df = df.iloc[:n]

    # Get the number of tokens and semantic complexity for the system message and user message and set a new column for each
    sys_msg_num_tokens, sys_msg_semantic_complexity = tokenize_text(sys_msg)
    usr_msg_num_tokens, usr_msg_semantic_complexity = tokenize_text(usr_msg)

    print(f"Number of tokens in user message: {usr_msg_num_tokens}")
    print(f"Average word length in user message: {usr_msg_semantic_complexity}")

    print(f"Number of tokens in system message: {sys_msg_num_tokens}")
    print(f"Average word length in system message: {sys_msg_semantic_complexity}")

    df['claim_num_tokens'] = df['claim'].apply(lambda x: tokenize_text(x)[0])
    df['claim_avg_wrd_len'] = df['claim'].apply(lambda x: tokenize_text(x)[1])

    print(f"Total number of claims: {len(df)}")
    print(f"Total number of tokens in claims: {df['claim_num_tokens'].sum()}")

    prompt = { 'system_message': sys_msg, 'user_message': usr_msg }

    df['model_prediction'] = df['claim'].apply(lambda x: fact_check_claim(x, temp, prompt))

    gpt4o_raw_acc = accuracy_score(df['label'], df['model_prediction'])
    gpt4o_raw_kappa = cohen_kappa_score(df['label'], df['model_prediction'])

    results = pd.DataFrame(columns=['temp', 'n', 'rand_state', 'acc', 'kappa', 'sys_msg', 'user_msg', 'sys_msg_num_tokens', 'sys_msg_avg_wrd_len', 'usr_msg_num_tokens', 'usr_msg_avg_wrd_len'])
    results.loc[0] = [temp, n, rand_state, gpt4o_raw_acc, gpt4o_raw_kappa, prompt['system_message'], prompt['user_message'], sys_msg_num_tokens, sys_msg_semantic_complexity, usr_msg_num_tokens, usr_msg_semantic_complexity]

    return results, df



In [ ]:
#%cd ..

/content/drive/MyDrive/SUNY_Poly_DSA598


In [ ]:
# os.mkdir('GPT_4o_mini_results')

In [ ]:
# Set the date of the test (today)
now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=-5)))
date_fmt = now.strftime("%m-%d")
print(date_fmt)

03-15


In [ ]:
# Determine test number from directory length
num_files = len(os.listdir('GPT_4o_mini_results/liar_test'))
print(f"Number of files in GPT_4o_mini_results: {num_files}")
run_count = (num_files // 2) + 1
run_count_str = str(run_count).zfill(3)
print(f"Run count: {run_count}")

liar_test_df = pd.DataFrame(liar_test)

# Set these
temp = .90
n = 65
rand_state = 100
sys_msg = "You are an investigative agent tasked with verifying the truthfulness of claims. If you predict correctly, you will save someone's life. If you predict incorrectly, someone will die."
user_msg = "Classify the truthfulness of the text out of the following categories: '1' (if the claim is true), '0' (if the claim is false). Think logically about the structure of the sentence; does it sound like a lie?"

gpt4oMini_raw_tests, results_df = run_gpt4o_raw(liar_test_df, user_msg, sys_msg, temp, n, rand_state)

# Set this
gpt4oMini_raw_tests['dataset_name'] = 'liar_test'

gpt4oMini_raw_tests.to_csv(f'GPT_4o_mini_results/liar_test/meta_{run_count_str}_{date_fmt}.csv', index=False)
results_df.to_csv(f'GPT_4o_mini_results/liar_test/results_{run_count_str}_{date_fmt}.csv', index=False)

print(f"Saved results to GPT_4o_mini_results/liar_test/{run_count_str}_{date_fmt}.csv")

gpt4oMini_raw_tests

Number of files in GPT_4o_mini_results: 48
Run count: 25
Prompting GPT with 65 claims, and the following parameters:
Temperature: 0.9
Random State: 100
Number of tokens in user message: 50
Average word length in user message: 3.4
Number of tokens in system message: 34
Average word length in system message: 4.5588235294117645
Total number of claims: 65
Total number of tokens in claims: 3824
Saved results to GPT_4o_mini_results/liar_test/025_03-15.csv


,temp,n,rand_state,acc,kappa,sys_msg,user_msg,sys_msg_num_tokens,sys_msg_avg_wrd_len,usr_msg_num_tokens,usr_msg_avg_wrd_len,dataset_name
0,0.9,65,100,0.553846,0.101953,You are an investigative agent tasked with ver...,Classify the truthfulness of the text out of t...,34,4.558824,50,3.4,liar_test


In [ ]:
# Load all the files starting with 'meta' in the GPT_4o_mini_results directory and plot the 'acc' column on the y  with  temp on the x
all_time_metrics = pd.DataFrame(columns=['temp', 'n', 'rand_state', 'acc', 'kappa', 'sys_msg', 'user_msg', 'model_id'])
for file in os.listdir('GPT_4o_mini_results/liar_test'):
    if file.startswith('meta'):
        df = pd.read_csv(os.path.join('GPT_4o_mini_results/liar_test', file))
        # Fill na in all colmumns with pd.NA
        df = df.fillna(pd.NA)
        df['temp'] = df['temp'].astype(float)
        df['n'] = df['n'].astype(int)
        df['model_id'] = file.split('_')[1]
        print(f"Processing {file}")
        all_time_metrics = pd.concat([all_time_metrics, df])

# Convert the 'n' column to an int
all_time_metrics['n'] = all_time_metrics['n'].astype(int)

# Plot the accuracy vs tempeture by model_id with plotly (scatter plot), size by 'n'
fig = px.scatter(all_time_metrics, x='temp', y='acc', color='model_id', size='n',
                 hover_data=['temp', 'acc', 'n'],
                 title='Accuracy vs Temperature by Model ID',
                 labels={'temp': 'Temperature', 'acc': 'Accuracy'},
                 color_discrete_sequence=px.colors.qualitative.Plotly)
# Update traces to make the points bigger and outlined in black
fig.update_traces(marker=dict(line=dict(width=2, color='black')), selector=dict(mode='markers'))
fig.update_layout(xaxis_title='Temperature', yaxis_title='Accuracy')
fig.show()

Processing meta_013_03-11.csv
Processing meta_005_03-11.csv
Processing meta_008_03-11.csv
Processing meta_010_03-11.csv
Processing meta_002_03-11.csv
Processing meta_006_03-11.csv
Processing meta_003_03-11.csv
Processing meta_009_03-11.csv
Processing meta_001_03-11.csv
Processing meta_007_03-11.csv
Processing meta_004_03-11.csv
Processing meta_012_03-11.csv
Processing meta_011_03-11.csv
Processing meta_014_03-12.csv
Processing meta_015_03-12.csv
Processing meta_016_03-12.csv
Processing meta_017_03-12.csv
Processing meta_018_03-12.csv
Processing meta_019_03-12.csv
Processing meta_020_03-12.csv
Processing meta_021_03-12.csv
Processing meta_022_03-12.csv
Processing meta_023_03-12.csv
Processing meta_024_03-12.csv
Processing meta_025_03-15.csv


<ipython-input-44-94b997c00e1a>:12: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [ ]:
# Get the best model by accuracy
# Sort the dataframe by acc
all_time_metrics = all_time_metrics.sort_values(by=['acc'], ascending=False)
# Print the first row
all_time_metrics.head(1)

,temp,n,rand_state,acc,kappa,sys_msg,user_msg,model_id,dataset_name,sys_msg_num_tokens,sys_msg_avg_wrd_len,usr_msg_num_tokens,usr_msg_avg_wrd_len
0,0.9,65,100,0.6,0.198292,NaN,Classify the truthfulness of the text out of t...,024,liar_test,0.0,0.0,50.0,3.4


In [ ]:
results_df['model_prediction'].unique()

array(['0', '1'], dtype=object)

In [ ]:
# Drop rows where 'model_prediction' is not 1 or 0
print(f"Length before dropping misclassifications: {len(liar_valid_df)}")
liar_valid_df = liar_valid_df[liar_valid_df['model_prediction'].isin(['1', '0'])]
print(f"Length after dropping misclassifications: {len(liar_valid_df)}")

Length before dropping misclassifications: 35
Length after dropping misclassifications: 35


In [ ]:
# Generate a classification report from the model's predictions
print(classification_report(results_df['label'], results_df['model_prediction']))


              precision    recall  f1-score   support

           0       0.48      0.54      0.51        28
           1       0.62      0.57      0.59        37

    accuracy                           0.55        65
   macro avg       0.55      0.55      0.55        65
weighted avg       0.56      0.55      0.56        65



### Fine-tune GPT-4o-mini on LIAR train

In [ ]:
train_path = 'LIAR/liar_train.jsonl'
val_path = 'LIAR/liar_valid.jsonl'

# Upload the training file
training_file = client.files.create(
    file=open(train_path, "rb"),
    purpose="fine-tune"
)

# Upload the validation file
validation_file = client.files.create(
    file=open(val_path, "rb"),
    purpose="fine-tune"
)

# Create the fine-tuning job
fine_tuning_job = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    validation_file=validation_file.id,
    model="gpt-4o-mini-2024-07-18",
    seed=2025,
)

print(f"Fine-tuning job started with ID: {fine_tuning_job.id}")
# ftjob-fnGzvRpBN2747hKB8vNQbBUR

Fine-tuning job started with ID: ftjob-QTgmjez0GvcpC2pc3JWH3RGb


In [ ]:

fine_tuning_job_id = "ftjob-QTgmjez0GvcpC2pc3JWH3RGb"

print(f"Monitoring fine-tuning job: {fine_tuning_job_id}")

while True:
    try:
        job_status = client.fine_tuning.jobs.retrieve(fine_tuning_job_id)
        status = job_status.status
        print(f"Job Status: {status}")

        if status in ["succeeded", "failed", "cancelled"]:
            if status == "succeeded":
                fine_tuned_model_id = job_status.fine_tuned_model
                print(f"Fine-tuning job succeeded! Fine-tuned model ID: {fine_tuned_model_id}")
            else:
                print(f"Fine-tuning job ended with status: {status}")
            break # Exit the loop when the job is done

        time.sleep(240) # Check status every 4 minutes

    except Exception as e:
        print(f"Error retrieving job status: {e}")
        break

Monitoring fine-tuning job: ftjob-QTgmjez0GvcpC2pc3JWH3RGb
Job Status: succeeded
Fine-tuning job succeeded! Fine-tuned model ID: ft:gpt-4o-mini-2024-07-18:personal::B8fgxGeb


### Evalute GPT-4o-mini's fact-checking capabilities on LIAR

In [ ]:
fine_tuning_job_id = "ftjob-QTgmjez0GvcpC2pc3JWH3RGb"
# Retrive the model and evaluate it
fine_tuned_model = client.fine_tuning.jobs.retrieve(fine_tuning_job_id).fine_tuned_model
print(f"Fine-tuned model name: {fine_tuned_model}")

# Retrieve fine-tuning job details
fine_tuning_job = client.fine_tuning.jobs.retrieve(fine_tuning_job_id)
print(f"Training Job Details:\n{fine_tuning_job}")

# FineTuningJob(id='ftjob-dkIRTFJOA6Qh11CdmZomcbzH', created_at=1733026293, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::AZWBdadz', finished_at=1733028567, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-kl11ePoaunzKiXcGWah4MsfM', result_files=['file-Q8EaHCC7AuVGiVSuheYoeS'], seed=42, status='succeeded', trained_tokens=126372, training_file='file-18zW8MGB58XZC5wR98V32N', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
result_file = fine_tuning_job.result_files[0]

Fine-tuned model name: ft:gpt-4o-mini-2024-07-18:personal::B8fgxGeb
Training Job Details:
FineTuningJob(id='ftjob-QTgmjez0GvcpC2pc3JWH3RGb', created_at=1741403526, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::B8fgxGeb', finished_at=1741406585, hyperparameters=Hyperparameters(batch_size=13, learning_rate_multiplier=1.8, n_epochs=2), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-kl11ePoaunzKiXcGWah4MsfM', result_files=['file-JFWHrVnqWXZfArATeBuiya'], seed=2025, status='succeeded', trained_tokens=1730122, training_file='file-6jg5yJTKdd6HUJx4dJCtth', validation_file='file-GLcbhmj4NmyPyvqBfXtiR7', estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=13, learning_rate_multiplier=1.8, n_epochs=2)), type='supervised'), user_provided_suffix=None, metadata=None)


In [ ]:
liar_test_df = pd.DataFrame(liar_test)
liar_test_df = liar_test_df[['messages']]
liar_test_df['claim'] = liar_test_df['messages'].apply(lambda x: x[1]['content'])
liar_test_df['label'] = liar_test_df['messages'].apply(lambda x: x[2]['content'])
liar_test_df = liar_test_df[['claim', 'label']]
liar_test_df['model_prediction'] = ''

In [ ]:
liar_test_df

,claim,label,model_prediction
0,Building a wall on the U.S.-Mexico border will...,1,
1,Wisconsin is on pace to double the number of l...,0,
2,Says John McCain has done nothing to help the ...,0,
3,Suzanne Bonamici supports a plan that will cut...,1,
4,When asked by a reporter whether hes at the ce...,0,
...,...,...,...
1262,Says his budget provides the highest state fun...,1,
1263,Ive been here almost every day.,0,
1264,"In the early 1980s, Sen. Edward Kennedy secret...",0,
1265,Says an EPA permit languished under Strickland...,0,


In [ ]:
model_id = 'ft:gpt-4o-mini-2024-07-18:personal::B8fgxGeb'

# Function to label data using any model
def label_data(df, model_name):
    labeled_data = []
    for _, row in df.iterrows():
        completion = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": "You are a fact checking classifier. Classify the truthfulness of the text out of the following categories: '1' (if the claim is true), '0' (if the claim is false). Do not use any other labels."},
                {"role": "user", "content": f"Classify the truthfulness of the text out of the following categories: '1' (if the claim is true), '0' (if the claim is false). Do not use any other labels.\n\n{row['claim']}"},
            ]
        )
        label = completion.choices[0].message.content
        labeled_data.append ({
            "claim": row['claim'],
            "label": row['label'],
            "model_prediction": label
        })
    return pd.DataFrame(labeled_data)
# Label the validation set using the fine-tuned model

liar_test_df = pd.DataFrame(liar_test)
liar_test_df = liar_test_df[['messages']]
liar_test_df['claim'] = liar_test_df['messages'].apply(lambda x: x[1]['content'])
liar_test_df['label'] = liar_test_df['messages'].apply(lambda x: x[2]['content'])
liar_test_df = liar_test_df[['claim', 'label']]
liar_test_df['model_prediction'] = ''

# Label the test set using the fine-tuned model
liar_test_df = label_data(liar_test_df, model_name=model_id)

In [ ]:
liar_test_df['model_prediction'].unique()

array(['1', '0'], dtype=object)

In [ ]:
print(classification_report(liar_test_df['label'], liar_test_df['model_prediction']))


              precision    recall  f1-score   support

           0       0.60      0.61      0.60       553
           1       0.69      0.69      0.69       714

    accuracy                           0.65      1267
   macro avg       0.65      0.65      0.65      1267
weighted avg       0.65      0.65      0.65      1267



```
              precision    recall  f1-score   support

           0       0.60      0.61      0.60       553
           1       0.69      0.69      0.69       714

    accuracy                           0.65      1267
   macro avg       0.65      0.65      0.65      1267
weighted avg       0.65      0.65      0.65      1267
```